### DMD Ingredient->Product Lookup

Get a list of all VMPs (Virtual Medicinal Products) and AMPs (Actual Medicinal Products) from a given list of ingredients (VTMs = Virtual Therapeutic Moieties)

In [1]:
# set name of codelist for exporting file
codelist_name="covid_meds"


# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Tocilizumab', 'Sarilumab']


from ebmdatalab import bq
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Select products from DMD containing chosen ingredients

### Dexamethasone

COVID treatment

Adult dosage
Dexamethasone:
• 6 mg orally once a day for 10 days (three 2 mg tablets or 15 ml of 2 mg/5 ml oral solution) or
• 6 mg intravenously once a day for 10 days (1.8 ml of 3.3 mg/ml ampoules [5.94 mg])
For people able to swallow and in whom there are no significant concerns about enteral absorption, prescribe tablets. Only use
intravenous administration when tablets or oral solutions are inappropriate or unavailable

In [5]:
# convert list to tuple for use in SQL query
names_tuple = tuple(names)
if len(names_tuple)==1:
    # remove comma if only one item
    names_tuple = str(names_tuple).replace(",","")

sql = f'''
SELECT "vmp" AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, ddd.ddd
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}
LEFT JOIN dmd.ddd on vmp.id=ddd.vpid

UNION ALL 

-- Dexamethasone selected preparations
SELECT "vmp" AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, ddd.ddd
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm LIKE "Dexamethasone%" -- include salts
LEFT JOIN dmd.ddd on vmp.id=ddd.vpid
WHERE 
vmp.nm LIKE '%2mg %tablets%' OR
vmp.nm LIKE '%2mg/5ml oral%' OR 
vmp.nm LIKE '%3.3mg/1ml solution for injection ampoules'

ORDER BY type, nm  '''

meds = bq.cached_read(sql, csv_path=os.path.join('..','..','data',f'meds_{codelist_name}.csv'))

meds.count()

type          22
id            22
bnf_code      15
nm            22
ingredient    22
ddd           15
dtype: int64

#### Check if any ingredients were not found

In [6]:
ings = list(meds.ingredient.drop_duplicates())
names.sort()
ings.sort()
print("Selected ingredients:", names)
print("Ingredients found:", ings)

missing_ings = [i for i in names if i not in ings]
print("Ingredients NOT found:", missing_ings)

Selected ingredients: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients found: ['Casirivimab', 'Dexamethasone', 'Dexamethasone sodium phosphate', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients NOT found: []


In [7]:
# check how many products have DDDs
print(meds[["ddd", "id"]].count())

ddd    15
id     22
dtype: int64


In [8]:
meds

,type,id,bnf_code,nm,ingredient,ddd
0,vmp,39657711000001106,None,Casirivimab 120mg/ml solution for infusion 11.1ml vials,Casirivimab,NaN
1,vmp,40035711000001105,None,Casirivimab 120mg/ml solution for infusion 2.5ml vials,Casirivimab,NaN
2,vmp,4052611000001105,0603020AAAAABAB,Dexamethasone (base) 3.3mg/1ml solution for injection ampoules,Dexamethasone sodium phosphate,1.5
3,vmp,32071811000001109,0603020G0AACFCF,Dexamethasone 2mg soluble tablets sugar free,Dexamethasone sodium phosphate,1.5
4,vmp,325356003,0603020G0AAADAD,Dexamethasone 2mg tablets,Dexamethasone,1.5
5,vmp,11716011000001103,0603020G0AAA6A6,Dexamethasone 2mg/5ml oral solution,Dexamethasone,1.5
6,vmp,37084211000001103,0603020G0AABXBX,Dexamethasone 2mg/5ml oral solution sugar free,Dexamethasone sodium phosphate,1.5
7,vmp,11716111000001102,0603020G0AAA6A6,Dexamethasone 2mg/5ml oral suspension,Dexamethasone,1.5
8,vmp,39658211000001100,None,Imdevimab 120mg/ml solution for infusion 11.1ml vials,Imdevimab,NaN
9,vmp,40035811000001102,None,Imdevimab 120mg/ml solution for infusion 2.5ml vials,Imdevimab,NaN
